#1. Data

1.1 Load

In [1]:
from sklearn.preprocessing import Normalizer, LabelEncoder
import pandas as pd
import numpy as np

df = pd.read_csv('motion_data.csv')
df.info()
df = df.sample(200)

y = df['Class']
enc = LabelEncoder()
y = enc.fit_transform(y)
print(y)
df = df.drop(columns=['Class', 'Timestamp'])

df.info()

classes = set(y)
print(df.head(5))
classes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3644 entries, 0 to 3643
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   AccX       3644 non-null   float64
 1   AccY       3644 non-null   float64
 2   AccZ       3644 non-null   float64
 3   GyroX      3644 non-null   float64
 4   GyroY      3644 non-null   float64
 5   GyroZ      3644 non-null   float64
 6   Class      3644 non-null   object 
 7   Timestamp  3644 non-null   int64  
dtypes: float64(6), int64(1), object(1)
memory usage: 227.9+ KB
[0 1 2 2 2 1 2 2 2 1 1 0 1 0 1 0 0 2 1 2 2 1 2 0 1 0 1 0 2 2 2 0 1 2 0 0 2
 2 0 2 0 2 0 2 2 2 0 1 0 2 2 0 2 2 0 0 2 2 0 2 0 0 1 0 0 1 1 2 2 1 2 0 2 2
 2 1 0 2 1 0 0 1 2 0 0 0 0 1 2 0 0 1 0 2 1 2 0 2 2 1 0 2 1 2 0 2 1 2 0 1 2
 2 0 1 0 1 2 1 1 1 2 0 0 0 1 2 1 1 2 2 0 0 0 0 1 0 0 2 1 2 0 1 0 0 0 0 0 2
 1 1 0 1 2 1 1 1 0 1 1 0 2 1 0 0 2 2 2 0 1 2 1 0 0 2 1 2 2 2 0 0 0 1 1 2 0
 0 0 1 0 0 2 0 1 0 2 0 1 2 1 2]
<class 'pandas.core.fra

{0, 1, 2}

1.2 Bootstrap

In [2]:
df['y_true'] = y
bts = []
for i in range(3):
  df_bt = df.sample(frac=0.5)
  y_bt = df_bt['y_true']
  X_bt = df_bt.drop(columns=['y_true'])
  bts.append([X_bt, y_bt])

1.3 Classification

In [3]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

models = [GaussianNB(), KNeighborsClassifier(), SVC()]
preds= []
for i, model in enumerate(models):
  X_bt = bts[i][0]
  y_bt = bts[i][1]
  model.fit(X_bt, y_bt)
  y_pred = model.predict(X_bt)
  pred_df = pd.DataFrame(y_pred, index = X_bt.index)
  preds.append(pred_df)
  print(accuracy_score(y_bt, y_pred))

0.5
0.59
0.57


1.4 Aggregation of predictions

In [4]:
preds_df = pd.concat(preds, axis=1)
preds_df.head()

,0,0,0
72,NaN,NaN,0.0
76,NaN,1.0,0.0
113,2.0,NaN,2.0
140,2.0,0.0,0.0
149,2.0,1.0,NaN


In [5]:
preds_df.to_csv('preds.csv')

#2. Ontology

2.1 Epistemic and doxastic logic

2.2 Creating ontology

In [6]:
!pip install owlready2
from owlready2 import *

onto = get_ontology('onto.owl')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


2.3 Creating classes and properties

In [7]:
import types

with onto:
  tsr = types.new_class('tsr', (ObjectProperty, TransitiveProperty, SymmetricProperty, ReflexiveProperty))
  ts = types.new_class('ts', (ObjectProperty, TransitiveProperty, SymmetricProperty))

  for cl in classes:
    onto_cl = types.new_class(f'Class{cl}', (Thing,))
    onto_cl_know = types.new_class(f'Know_Class{cl}', (Thing,))
    onto_cl_know.equivalent_to.append(tsr.only(onto_cl))
    onto_cl_believe = types.new_class(f'Believe_Class{cl}', (Thing,))
    onto_cl_believe.equivalent_to.append(ts.only(onto_cl))

onto.save('onto.owl', format='ntriples')

2.4 Creating instances

In [8]:
with onto:
  for index, row in preds_df.iterrows():
    inst_old = None
    for j, pred in enumerate(preds_df.columns):
      cl = row.iloc[j]
      if not pd.isna(cl):
        cl = int(cl)
        onto_cl = onto[f'Class{cl}']
        inst_new = onto_cl(f'inst{index}_{j}')
        if inst_old != None:
          inst_old.tsr.append(inst_new)
          inst_old.ts.append(inst_new)
        inst_old = inst_new

onto.save('onto_instances.owl', format='ntriples')

2.5 Inferring relations

In [9]:
with onto:
  sync_reasoner_pellet(infer_property_values=True, debug=2)

onto.save('onto_relations.owl', format='ntriples')

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /usr/local/lib/python3.8/dist-packages/owlready2/pellet/antlr-runtime-3.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-tdb-0.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-iri-0.9.5.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/httpcore-4.2.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/aterm-java-1.6.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/pellet-2.3.1.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/log4j-1.2.16.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-core-2.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/slf4j-log4j12-1.6.4.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/commons-codec-1.6.jar:/usr/local/lib/py

* Owlready * Adding relation onto.inst3136_1 tsr onto.inst3136_1
* Owlready * Adding relation onto.inst2804_2 tsr onto.inst2804_2
* Owlready * Adding relation onto.inst909_1 tsr onto.inst909_1
* Owlready * Adding relation onto.inst909_1 ts onto.inst909_1
* Owlready * Adding relation onto.inst2448_0 tsr onto.inst2448_2
* Owlready * Adding relation onto.inst2448_0 tsr onto.inst2448_0
* Owlready * Adding relation onto.inst2448_0 ts onto.inst2448_2
* Owlready * Adding relation onto.inst2448_0 ts onto.inst2448_0
* Owlready * Adding relation onto.inst2691_1 tsr onto.inst2691_1
* Owlready * Adding relation onto.inst2691_1 ts onto.inst2691_1
* Owlready * Adding relation onto.inst2067_1 tsr onto.inst2067_1
* Owlready * Adding relation onto.inst1236_0 tsr onto.inst1236_2
* Owlready * Adding relation onto.inst1236_0 tsr onto.inst1236_0
* Owlready * Adding relation onto.inst1236_0 ts onto.inst1236_2
* Owlready * Adding relation onto.inst1236_0 ts onto.inst1236_0
* Owlready * Adding relation onto.i

* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


2.6 OWA/CWA

In [10]:
with onto:
  for inst in onto.individuals():
    close_world(inst)

2.7 Inferring classification

In [11]:
with onto:
  sync_reasoner_pellet(debug=2)

onto.save('onto_reasoned.owl', format='ntriples')

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /usr/local/lib/python3.8/dist-packages/owlready2/pellet/antlr-runtime-3.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-tdb-0.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-iri-0.9.5.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/httpcore-4.2.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/aterm-java-1.6.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/pellet-2.3.1.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/log4j-1.2.16.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-core-2.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/slf4j-log4j12-1.6.4.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/commons-codec-1.6.jar:/usr/local/lib/py